# Generate LPCs From Audio Files

In [19]:
import librosa
import numpy as np

## Load In Audio Files into Memory

In [20]:
audio_file, _ = librosa.load("./wav_data/Vowels_Amanda/aa_amanda.wav")

In [24]:
lpc_vals = librosa.lpc(audio_file, order=14)


In [26]:
print(lpc_vals)
len(lpc_vals)

[ 1.         -2.2317953   1.5848887  -0.55056894  0.5436697  -0.337158
 -0.4170133   0.61259896  0.05084044 -0.48304117  0.6448273  -0.65520066
  0.2798591  -0.08932586  0.06248806]


15